# Data preprocessing and integration

## Data confirm

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce

In [47]:
data_dir = "../data/"

In [48]:
tax_df = pd.read_csv(data_dir + "OECD_PWC_corporate_tax_rates_2013_2022.csv")
gerd_df = pd.read_csv(data_dir + "Unesco_GERD as per percentage of GDP.csv")
gii_df = pd.read_csv(data_dir + "WIPO_GII_paneldata.csv")
patent_df = pd.read_csv(data_dir + "WIPO_Patent publications for AI-related technology.csv")
wipo_tax_df = pd.read_csv(data_dir + "WIPO_Tax.csv")
gdp_df = pd.read_csv(data_dir + "WorldBank_GDP.csv")
revenue_df = pd.read_csv(data_dir + "WorldBank_General Revenue.csv")
internet_df = pd.read_csv(data_dir + "WorldBank_Individuals using the Internet (of population).csv")
unemployment_df = pd.read_csv(data_dir + "WorldBank_Unemployment_Rate.csv")

In [49]:
tax_df.columns

Index(['연도', '한국', '일본', '중국', '독일', '영국', '미국', '프랑스'], dtype='object')

In [50]:
gerd_df.columns

Index(['Country', 'year', 'value'], dtype='object')

In [51]:
gii_df.columns

Index(['Economies', 'Year', 'Institutions', 'Human capital and research',
       'Infrastructure', 'Market sophistication', 'Business sophistication',
       'Knowledge and technology outputs', 'Creative outputs',
       'Global Innovation Index', 'GNI per capita in current U.S. dollars ',
       'GDP per capita ppp (current international $)'],
      dtype='object')

In [52]:
patent_df.columns

Index(['Office', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020', '2021', '2022'],
      dtype='object')

In [53]:
wipo_tax_df.columns

Index(['Reference area', 'TIME_PERIOD', 'OBS_VALUE'], dtype='object')

In [54]:
gdp_df.columns

Index(['Country Name', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020', '2021', '2022'],
      dtype='object')

In [55]:
revenue_df.columns

Index(['Country Name', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020', '2021', '2022'],
      dtype='object')

In [56]:
internet_df.columns

Index(['Country Name', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020', '2021', '2022'],
      dtype='object')

In [57]:
unemployment_df.columns

Index(['Country Name', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020', '2021', '2022'],
      dtype='object')

In [58]:
# 국가명 통일을 위한 매핑
country_mapping = {
    '한국': 'Korea',
    '일본': 'Japan',
    '중국': 'China',
    '독일': 'Germany',
    '영국': 'United Kingdom',
    '미국': 'United States of America',
    '프랑스': 'France',
    'Korea, Rep.': 'Korea',
    'United States': 'United States of America'
}

## long format 데이터를 pivot하여 원하는 형식으로 변환

In [59]:
tax_df.rename(columns={'연도': 'Year'}, inplace=True)
tax_long = tax_df.melt(id_vars=['Year'], var_name='Country', value_name='Corporate_Tax')
tax_long['Country'] = tax_long['Country'].map(country_mapping)
tax_long['Year'] = tax_long['Year'].astype(int)

def pivot_wide(df, value_col, category_label):
    pivot_df = df.pivot(index='Country', columns='Year', values=value_col)
    pivot_df.reset_index(inplace=True)
    pivot_df.insert(1, 'category', category_label)
    year_cols = [year for year in range(2013, 2023) if year in pivot_df.columns]
    pivot_df = pivot_df[['Country', 'category'] + year_cols]
    return pivot_df

tax_wide = pivot_wide(tax_long, 'Corporate_Tax', 'Corporate Tax')

In [60]:
print(tax_wide['Country'].unique())
print(tax_wide)

['China' 'France' 'Germany' 'Japan' 'Korea' 'United Kingdom'
 'United States of America']
Year                   Country       category   2013   2014   2015   2016  \
0                        China  Corporate Tax  25.0%  25.0%  25.0%  25.0%   
1                       France  Corporate Tax  38.0%  38.0%  38.0%  34.4%   
2                      Germany  Corporate Tax  29.6%  29.7%  29.8%  29.8%   
3                        Japan  Corporate Tax  37.0%  34.6%  32.1%  30.0%   
4                        Korea  Corporate Tax  24.2%  24.2%  24.2%  24.2%   
5               United Kingdom  Corporate Tax  23.0%  21.0%  20.0%  20.0%   
6     United States of America  Corporate Tax  39.0%  39.1%  39.0%  38.9%   

Year   2017   2018   2019   2020   2021   2022  
0     25.0%  25.0%  25.0%  25.0%  25.0%  25.0%  
1     44.4%  34.4%  34.4%  32.0%  28.4%  25.8%  
2     29.9%  29.9%  29.9%  29.8%  29.9%  29.9%  
3     30.0%  29.7%  29.7%  29.7%  29.7%  29.7%  
4     24.2%  27.5%  27.5%  27.5%  27.5%  27.5%  

## 데이터셋 전처리 후 pivot

In [61]:
# 2. Unesco_GERD as per percentage of GDP.csv (gerd_df)
gerd_df = pd.read_csv(data_dir + "Unesco_GERD as per percentage of GDP.csv")
gerd_df.rename(columns={'year': 'Year', 'value': 'GERD'}, inplace=True)
gerd_df['Year'] = gerd_df['Year'].astype(int)
gerd_df['Country'] = gerd_df['Country'].replace({'United State of America': 'United States of America'})
gerd_wide = pivot_wide(gerd_df, 'GERD', 'GERD')

print(gerd_wide['Country'].unique())
print(gerd_wide)

['China' 'France' 'Germany' 'Japan' 'Korea' 'United Kingdom'
 'United States of America']
Year                   Country category     2013     2014     2015     2016  \
0                        China     GERD  1.99786  2.02243  2.05701  2.10033   
1                       France     GERD  2.23703  2.27592  2.22702  2.22238   
2                      Germany     GERD  2.83599  2.87784  2.93379  2.94039   
3                        Japan     GERD  3.27896  3.36788  3.24071  3.10666   
4                        Korea     GERD  3.95124  4.07786  3.97820  3.98704   
5               United Kingdom     GERD  1.62096  2.26488  2.27566  2.32052   
6     United States of America     GERD  2.69592  2.70881  2.77328  2.83676   

Year     2017     2018     2019     2020     2021     2022  
0     2.11603  2.14058  2.24463  2.40666  2.43260  2.55518  
1     2.19888  2.19666  2.19179  2.27464  2.21808  2.23312  
2     3.04710  3.11011  3.16701  3.13136  3.12882  3.13236  
3     3.16636  3.21920  3.21824  

In [62]:
# 3. WIPO_GII_paneldata.csv (gii_df)
gii_df = pd.read_csv(data_dir + "WIPO_GII_paneldata.csv")
gii_df.rename(columns={
    'Economies': 'Country',
    'GNI per capita in current U.S. dollars ': 'GNI_per_capita',
    'GDP per capita ppp (current international $)': 'GDP_per_capita_PPP'
}, inplace=True)
gii_df['Year'] = gii_df['Year'].astype(int)
gii_indicators = ['Institutions', 'Human capital and research', 'Infrastructure', 
                  'Market sophistication', 'Business sophistication', 
                  'Knowledge and technology outputs', 'Creative outputs', 
                  'Global Innovation Index', 'GNI_per_capita', 'GDP_per_capita_PPP']
gii_wide_list = []
for indicator in gii_indicators:
    temp = gii_df[['Country', 'Year', indicator]].copy()
    temp.rename(columns={indicator: 'Value'}, inplace=True)
    temp_wide = pivot_wide(temp, 'Value', indicator)
    gii_wide_list.append(temp_wide)
gii_wide = pd.concat(gii_wide_list, axis=0, ignore_index=True)

print(gii_wide['Country'].unique())
print(gii_wide)

['China' 'France' 'Germany' 'Japan' 'Korea' 'United Kingdom'
 'United States of America']
Year                   Country            category         2013         2014  \
0                        China        Institutions     48.30000     48.30000   
1                       France        Institutions     79.00000     78.60000   
2                      Germany        Institutions     82.50000     82.70000   
3                        Japan        Institutions     83.50000     84.10000   
4                        Korea        Institutions     76.00000     75.80000   
..                         ...                 ...          ...          ...   
65                     Germany  GDP_per_capita_PPP  44993.89275  47011.55109   
66                       Japan  GDP_per_capita_PPP  39402.02510  39555.41201   
67                       Korea  GDP_per_capita_PPP  34244.31214  35324.49767   
68              United Kingdom  GDP_per_capita_PPP  40218.74726  41580.60249   
69    United States of America

In [63]:
# 4. WIPO_Patent publications for AI-related technology.csv (patent_df)

patent_df = pd.read_csv(data_dir + "WIPO_Patent publications for AI-related technology.csv")
patent_df.rename(columns={'Office': 'Country'}, inplace=True)
patent_long = patent_df.melt(id_vars=['Country'], var_name='Year', value_name='Patent_Publications')
patent_long['Country'] = patent_long['Country'].replace({'Republic of Korea': 'Korea'})
patent_long['Year'] = patent_long['Year'].astype(int)
patent_wide = pivot_wide(patent_long, 'Patent_Publications', 'Patent Publications')
print(patent_wide['Country'].unique())
print(patent_wide)

['China' 'France' 'Germany' 'Japan' 'Korea' 'United Kingdom'
 'United States of America']
Year                   Country             category   2013   2014   2015  \
0                        China  Patent Publications  10367  12334  14812   
1                       France  Patent Publications    155    181    192   
2                      Germany  Patent Publications    346    379    519   
3                        Japan  Patent Publications   9131   8797   9171   
4                        Korea  Patent Publications   4601   5847   5922   
5               United Kingdom  Patent Publications    253    309    229   
6     United States of America  Patent Publications  17768  20943  22825   

Year   2016   2017   2018   2019   2020   2021   2022  
0     16214  20372  19480  19598  20073  22475  23598  
1       243    217    196    227    202    181    159  
2       575    525    548    473    501    448    434  
3      8542   8852   6998   6982   6807   6117   5218  
4      5548   5268   

In [64]:
# 5. WIPO_Tax.csv (wipo_tax_df)
wipo_tax_df = pd.read_csv(data_dir + "WIPO_Tax.csv")
wipo_tax_df.rename(columns={'Reference area': 'Country', 'TIME_PERIOD': 'Year', 'OBS_VALUE': 'WIPO_Tax'}, inplace=True)
wipo_tax_df['Year'] = wipo_tax_df['Year'].astype(int)
wipo_tax_df['Country'] = wipo_tax_df['Country'].replace({'United States': 'United States of America'})
wipo_tax_wide = pivot_wide(wipo_tax_df, 'WIPO_Tax', 'WIPO Tax')

print(wipo_tax_wide['Country'].unique())
print(wipo_tax_wide)

['China' 'France' 'Germany' 'Japan' 'Korea' 'United Kingdom'
 'United States of America']
Year                   Country  category    2013    2014    2015    2016  \
0                        China  WIPO Tax  18.640  18.518  18.135  17.464   
1                       France  WIPO Tax  45.329  45.406  45.363  45.397   
2                      Germany  WIPO Tax  37.675  37.614  38.052  38.518   
3                        Japan  WIPO Tax  28.557  29.973  30.242  30.279   
4                        Korea  WIPO Tax  23.143  23.381  23.737  24.745   
5               United Kingdom  WIPO Tax  32.007  31.693  31.907  32.450   
6     United States of America  WIPO Tax  25.413  25.786  26.087  25.713   

Year    2017    2018    2019    2020    2021    2022  
0     17.351  17.014  22.089  20.077  21.040  20.050  
1     46.239  46.022  45.024  45.239  45.059  45.774  
2     38.528  39.196  39.326  38.652  39.781  39.647  
3     30.916  31.548  31.507  32.909  33.883  34.441  
4     25.357  26.686  27.2

In [65]:
# 6. World Bank 데이터: GDP, General Revenue, Internet Usage, Unemployment Rate

def process_wb(file_name, indicator_name):
    df = pd.read_csv(data_dir + file_name)
    df.rename(columns={'Country Name': 'Country'}, inplace=True)
    years = [str(y) for y in range(2013, 2023)]
    df_long = pd.melt(df, id_vars=['Country'], value_vars=years, var_name='Year', value_name=indicator_name)
    df_long['Country'] = df_long['Country'].replace({'United States': 'United States of America'})
    df_long['Year'] = df_long['Year'].astype(int)
    return df_long

In [66]:
gdp_long = process_wb("WorldBank_GDP.csv", "GDP")
gdp_wide = pivot_wide(gdp_long, 'GDP', 'GDP')

revenue_long = process_wb("WorldBank_General Revenue.csv", "General_Revenue")
revenue_wide = pivot_wide(revenue_long, 'General_Revenue', 'General Revenue')

internet_long = process_wb("WorldBank_Individuals using the Internet (of population).csv", "Internet_Usage")
internet_wide = pivot_wide(internet_long, 'Internet_Usage', 'Internet Usage')

unemployment_long = process_wb("WorldBank_Unemployment_Rate.csv", "Unemployment_Rate")
unemployment_wide = pivot_wide(unemployment_long, 'Unemployment_Rate', 'Unemployment Rate')

In [67]:
print(gdp_wide['Country'].unique())
print(gdp_wide)

['China' 'France' 'Germany' 'Japan' 'Korea' 'United Kingdom'
 'United States of America']
Year                   Country category          2013          2014  \
0                        China      GDP  9.570470e+12  1.047560e+13   
1                       France      GDP  2.816080e+12  2.861240e+12   
2                      Germany      GDP  3.808090e+12  3.965800e+12   
3                        Japan      GDP  5.212330e+12  4.896990e+12   
4                        Korea      GDP  1.370630e+12  1.484490e+12   
5               United Kingdom      GDP  2.784850e+12  3.064710e+12   
6     United States of America      GDP  1.688070e+13  1.760810e+13   

Year          2015          2016          2017          2018          2019  \
0     1.106160e+13  1.123330e+13  1.231050e+13  1.389490e+13  1.428000e+13   
1     2.442480e+12  2.470410e+12  2.588870e+12  2.781580e+12  2.722790e+12   
2     3.423570e+12  3.537780e+12  3.763090e+12  4.052010e+12  3.957210e+12   
3     4.444930e+12  5.003680e

In [68]:
print(revenue_wide['Country'].unique())
print(revenue_wide)

['China' 'France' 'Germany' 'Japan' 'Korea' 'United Kingdom'
 'United States of America']
Year                   Country         category       2013       2014  \
0                        China  General Revenue  10.956237  15.818178   
1                       France  General Revenue  44.111404  44.220335   
2                      Germany  General Revenue  27.991581  27.949899   
3                        Japan  General Revenue  36.051493  36.085117   
4                        Korea  General Revenue  26.115130  24.988444   
5               United Kingdom  General Revenue  33.726067  32.883296   
6     United States of America  General Revenue  18.740712  18.858845   

Year       2015       2016       2017       2018       2019       2020  \
0     16.043023  15.629926  15.883172  16.535896  16.100570  14.086087   
1     43.977573  43.903198  44.469026  44.163770  43.056331  42.826502   
2     27.956606  27.859773  28.008141  28.384614  28.666970  28.574162   
3     35.967089  35.881486  3

In [69]:
print(internet_wide['Country'].unique())
print(internet_wide)

['China' 'France' 'Germany' 'Japan' 'Korea' 'United Kingdom'
 'United States of America']
Year                   Country        category     2013     2014     2015  \
0                        China  Internet Usage  45.8000  47.9000  50.3000   
1                       France  Internet Usage  81.9198  83.7511  78.0060   
2                      Germany  Internet Usage  84.1700  86.1937  87.5898   
3                        Japan  Internet Usage  88.2194  89.1068  91.0580   
4                        Korea  Internet Usage  84.7700  87.5568  89.8963   
5               United Kingdom  Internet Usage  89.8441  91.6133  92.0003   
6     United States of America  Internet Usage  71.4000  73.0000  74.5542   

Year     2016     2017     2018     2019     2020     2021     2022  
0     53.2000  54.3000  59.2000  64.0809  70.0528  73.0532  75.6113  
1     79.2698  80.5025  82.0432  83.3397  84.7064  86.0955  85.3333  
2     84.1652  84.3942  87.0371  88.1345  89.8129  91.4306  91.6298  
3     93.1827

In [70]:
print(unemployment_wide['Country'].unique())
print(unemployment_wide)

['China' 'France' 'Germany' 'Japan' 'Korea' 'United Kingdom'
 'United States of America']
Year                   Country           category   2013    2014    2015  \
0                        China  Unemployment Rate  4.600   4.630   4.650   
1                       France  Unemployment Rate  9.913  10.273  10.354   
2                      Germany  Unemployment Rate  5.316   4.979   4.612   
3                        Japan  Unemployment Rate  4.038   3.589   3.385   
4                        Korea  Unemployment Rate  2.747   3.082   3.546   
5               United Kingdom  Unemployment Rate  7.704   6.363   5.517   
6     United States of America  Unemployment Rate  7.375   6.168   5.280   

Year    2016   2017   2018   2019   2020   2021   2022  
0      4.560  4.470  4.310  4.560  5.000  4.550  4.980  
1     10.057  9.409  9.018  8.415  8.009  7.874  7.308  
2      4.104  3.781  3.384  3.163  3.881  3.594  3.120  
3      3.132  2.822  2.467  2.351  2.809  2.828  2.600  
4      3.650  3.

## 통합: 모든 pivoted 데이터를 세로로 concat

In [71]:
all_dfs = [
    tax_wide,
    gerd_wide,
    gii_wide,
    patent_wide,
    wipo_tax_wide,
    gdp_wide,
    revenue_wide,
    internet_wide,
    unemployment_wide
]

final_df = pd.concat(all_dfs, axis=0, ignore_index=True)

In [72]:
final_df.shape
print(final_df)

Year                   Country           category   2013   2014   2015   2016  \
0                        China      Corporate Tax  25.0%  25.0%  25.0%  25.0%   
1                       France      Corporate Tax  38.0%  38.0%  38.0%  34.4%   
2                      Germany      Corporate Tax  29.6%  29.7%  29.8%  29.8%   
3                        Japan      Corporate Tax  37.0%  34.6%  32.1%  30.0%   
4                        Korea      Corporate Tax  24.2%  24.2%  24.2%  24.2%   
..                         ...                ...    ...    ...    ...    ...   
121                    Germany  Unemployment Rate  5.316  4.979  4.612  4.104   
122                      Japan  Unemployment Rate  4.038  3.589  3.385  3.132   
123                      Korea  Unemployment Rate  2.747  3.082  3.546   3.65   
124             United Kingdom  Unemployment Rate  7.704  6.363  5.517  4.867   
125   United States of America  Unemployment Rate  7.375  6.168   5.28  4.869   

Year   2017   2018   2019  

In [73]:
print(final_df['category'].unique())
print(final_df["Country"].unique())

['Corporate Tax' 'GERD' 'Institutions' 'Human capital and research'
 'Infrastructure' 'Market sophistication' 'Business sophistication'
 'Knowledge and technology outputs' 'Creative outputs'
 'Global Innovation Index' 'GNI_per_capita' 'GDP_per_capita_PPP'
 'Patent Publications' 'WIPO Tax' 'GDP' 'General Revenue' 'Internet Usage'
 'Unemployment Rate']
['China' 'France' 'Germany' 'Japan' 'Korea' 'United Kingdom'
 'United States of America']


In [74]:
final_df.to_csv(data_dir + "no.csv", index=False)

In [75]:
data_path = "../data/no.csv"
df = pd.read_csv(data_path)

In [76]:
years = [str(year) for year in range(2013, 2023)]
mask = df['category'] == 'Corporate Tax'
for year in years:
    df.loc[mask, year] = df.loc[mask, year].astype(str).str.replace('%', '', regex=False).str.strip().astype(float)

In [77]:
for year in years:
    df[year] = pd.to_numeric(df[year], errors='coerce').round(4)
print(df[years].dtypes)

2013    float64
2014    float64
2015    float64
2016    float64
2017    float64
2018    float64
2019    float64
2020    float64
2021    float64
2022    float64
dtype: object


In [78]:
df.to_csv(data_dir + "master_data_by_category.csv", index=False)